In [4]:
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd

In [5]:
player_position_df = pd.read_csv('data/nfl-big-data-bowl-2024/players_basic.csv')
player_position_df.head()

,position,displayName
0,QB,Tom Brady
1,T,Jason Peters
2,QB,Aaron Rodgers
3,TE,Marcedes Lewis
4,QB,Matt Ryan


In [6]:
matchups_df = pd.read_csv('data/nfl-big-data-bowl-2024/matchups.csv')
matchups_df.head()

,matchup,gameId,week,home,away
0,LA_BUF,2022090800,1,LA,BUF
1,ATL_NO,2022091100,1,ATL,NO
2,CAR_CLE,2022091101,1,CAR,CLE
3,CHI_SF,2022091102,1,CHI,SF
4,CIN_PIT,2022091103,1,CIN,PIT


In [7]:
plays_df = pd.read_csv('data/nfl-big-data-bowl-2024/plays.csv')
plays_df.head()

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN


In [8]:
plays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12486 entries, 0 to 12485
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   gameId                            12486 non-null  int64  
 1   playId                            12486 non-null  int64  
 2   ballCarrierId                     12486 non-null  int64  
 3   ballCarrierDisplayName            12486 non-null  object 
 4   playDescription                   12486 non-null  object 
 5   quarter                           12486 non-null  int64  
 6   down                              12486 non-null  int64  
 7   yardsToGo                         12486 non-null  int64  
 8   possessionTeam                    12486 non-null  object 
 9   defensiveTeam                     12486 non-null  object 
 10  yardlineSide                      12319 non-null  object 
 11  yardlineNumber                    12486 non-null  int64  
 12  game

In [9]:
plays_df = pd.merge(player_position_df, plays_df, how='left', left_on='displayName', right_on='ballCarrierDisplayName')

In [10]:
plays_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13960 entries, 0 to 13959
Data columns (total 37 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   position                          13960 non-null  object 
 1   displayName                       13960 non-null  object 
 2   gameId                            12763 non-null  float64
 3   playId                            12763 non-null  float64
 4   ballCarrierId                     12763 non-null  float64
 5   ballCarrierDisplayName            12763 non-null  object 
 6   playDescription                   12763 non-null  object 
 7   quarter                           12763 non-null  float64
 8   down                              12763 non-null  float64
 9   yardsToGo                         12763 non-null  float64
 10  possessionTeam                    12763 non-null  object 
 11  defensiveTeam                     12763 non-null  object 
 12  yard

In [11]:
def clean_data(plays_df):
    # Drop unwanted columns
    plays_df = plays_df.drop(columns=['foulNFLId2', 'passProbability', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded', 'foulName1', 'foulName2', 'foulNFLId1', 'prePenaltyPlayResult', 'playNullifiedByPenalty'])

    # Convert columns to appropriate data types
    plays_df = plays_df.astype({'position': 'string', 'displayName': 'string', 'ballCarrierDisplayName': 'string', 'playDescription': 'string', 'possessionTeam': 'string', 'defensiveTeam': 'string', 'yardlineSide': 'string', 'offenseFormation': 'string'})
    plays_df['defendersInTheBox'] = plays_df['defendersInTheBox'].fillna(0).astype(int)

    # Change column type to datetime64[ns] for column: 'gameClock'
    plays_df['gameClock'] = pd.to_datetime(plays_df['gameClock'])

    # Convert selected columns to int64
    plays_df = plays_df.astype({'gameId': 'Int64', 'playId': 'Int64', 'ballCarrierId': 'Int64', 'quarter': 'Int64', 'down': 'Int64', 'yardsToGo': 'Int64', 'yardlineNumber': 'Int64', 'preSnapHomeScore': 'Int64', 'preSnapVisitorScore': 'Int64', 'playResult': 'Int64', 'absoluteYardlineNumber': 'Int64'})

    return plays_df

plays_df_clean = clean_data(plays_df.copy())
plays_df_clean.reset_index(drop=True, inplace=True)
plays_df_clean.head()


,position,displayName,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,...,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,playResult,absoluteYardlineNumber,offenseFormation,defendersInTheBox
0,QB,Tom Brady,2022102301,2725,25511,Tom Brady,"(:27) (No Huddle, Shotgun) T.Brady scrambles u...",3,2,8,...,2024-03-18 00:27:00,14,0,R,NaN,NaN,1,92,SHOTGUN,7
1,QB,Tom Brady,2022092512,2687,25511,Tom Brady,(1:08) (Shotgun) T.Brady scrambles up the midd...,3,3,3,...,2024-03-18 01:08:00,6,14,R,NaN,-10.0,-10,80,EMPTY,5
2,QB,Tom Brady,2022101607,3652,25511,Tom Brady,(7:07) T.Brady up the middle to PIT 29 for no ...,4,3,1,...,2024-03-18 07:07:00,20,12,NaN,NaN,NaN,0,81,SINGLEBACK,7
3,QB,Tom Brady,2022102700,3083,25511,Tom Brady,(13:22) (Shotgun) T.Brady scrambles left end t...,4,3,10,...,2024-03-18 13:22:00,10,17,R,NaN,NaN,1,23,SHOTGUN,5
4,T,Jason Peters,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaT,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,<NA>,0


In [12]:
plays_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13960 entries, 0 to 13959
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   position                13960 non-null  string        
 1   displayName             13960 non-null  string        
 2   gameId                  12763 non-null  Int64         
 3   playId                  12763 non-null  Int64         
 4   ballCarrierId           12763 non-null  Int64         
 5   ballCarrierDisplayName  12763 non-null  string        
 6   playDescription         12763 non-null  string        
 7   quarter                 12763 non-null  Int64         
 8   down                    12763 non-null  Int64         
 9   yardsToGo               12763 non-null  Int64         
 10  possessionTeam          12763 non-null  string        
 11  defensiveTeam           12763 non-null  string        
 12  yardlineSide            12594 non-null  string

In [14]:
plays_df_clean = plays_df_clean[[ 'gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName', 'position','playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'passLength', 'playResult','absoluteYardlineNumber', 'offenseFormation', 'defendersInTheBox']]


In [15]:
# Filter rows based on column: 'down'
plays_3rd_down_df = plays_df_clean[plays_df_clean['down'] == 3]
plays_3rd_down_df.head()
    

,gameId,playId,ballCarrierId,ballCarrierDisplayName,position,playDescription,quarter,down,yardsToGo,possessionTeam,...,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,playResult,absoluteYardlineNumber,offenseFormation,defendersInTheBox
1,2022092512,2687,25511,Tom Brady,QB,(1:08) (Shotgun) T.Brady scrambles up the midd...,3,3,3,TB,...,30,2024-03-18 01:08:00,6,14,R,NaN,-10,80,EMPTY,5
2,2022101607,3652,25511,Tom Brady,QB,(7:07) T.Brady up the middle to PIT 29 for no ...,4,3,1,TB,...,29,2024-03-18 07:07:00,20,12,NaN,NaN,0,81,SINGLEBACK,7
3,2022102700,3083,25511,Tom Brady,QB,(13:22) (Shotgun) T.Brady scrambles left end t...,4,3,10,TB,...,13,2024-03-18 13:22:00,10,17,R,NaN,1,23,SHOTGUN,5
5,2022110603,3064,29851,Aaron Rodgers,QB,(12:24) (Shotgun) A.Rodgers scrambles up the m...,4,3,17,GB,...,36,2024-03-18 12:24:00,15,6,R,NaN,18,46,SHOTGUN,4
7,2022091812,2969,29851,Aaron Rodgers,QB,(6:38) (Shotgun) A.Rodgers right end to GB 13 ...,4,3,2,GB,...,9,2024-03-18 06:38:00,24,10,NaN,NaN,4,101,SHOTGUN,6


In [16]:
plays_4th_down_df = plays_df_clean[plays_df_clean['down'] == 4]
plays_4th_down_df.head()

,gameId,playId,ballCarrierId,ballCarrierDisplayName,position,playDescription,quarter,down,yardsToGo,possessionTeam,...,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,playResult,absoluteYardlineNumber,offenseFormation,defendersInTheBox
14,2022092502,3467,33084,Matt Ryan,QB,(6:39) M.Ryan up the middle to IND 35 for 2 ya...,4,4,1,IND,...,33,2024-03-18 06:39:00,13,17,NaN,NaN,2,43,SINGLEBACK,8
71,2022102700,4249,37079,Julio Jones,WR,(:53) (Shotgun) T.Brady pass short right to J....,4,4,8,TB,...,8,2024-03-18 00:53:00,16,27,C,3.0,8,18,SHOTGUN,5
205,2022103000,1533,38605,Russell Wilson,QB,(5:23) R.Wilson up the middle to JAX 34 for 2 ...,2,4,1,DEN,...,36,2024-03-18 05:23:00,10,0,NaN,NaN,2,46,SINGLEBACK,7
209,2022103000,4058,38605,Russell Wilson,QB,(:36) R.Wilson right guard to JAX 25 for 1 yar...,4,4,1,DEN,...,26,2024-03-18 00:36:00,17,21,NaN,NaN,1,84,I_FORM,10
250,2022102304,4412,38696,Marvin Jones,WR,(:22) (Shotgun) T.Lawrence pass deep middle to...,4,4,15,JAX,...,40,2024-03-18 00:22:00,17,23,C,22.0,43,50,EMPTY,4
